In [2]:
from qiskit import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from qiskit.ignis.characterization.gates import *
from qiskit.providers.aer import *
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import *


In [ ]:
def ampcal_circuits(max_reps, qubits, dth=0.01):
    """
    Generates circuit for measuring the amplitude error of
    the single qubit gates

    The U2 gate is repeatedly applied (in groups of 2)
    and we look at the population of the
    qubit in the xy axis (amplitude erorr amplification sequence)

    Y90-(Y90-Y90)^n

    Args:
        max_reps (int): the maximum number of repetitions. Circuits will
            increment by 1 rep up to max_rep
        qubits (list): a list of integers indices of the qubits to perform the
            calibration on
    Returns:
        tuple: A tuple of the form (``circuits``, ``xdata``) where
            ``circuits`` is a list of QuantumCircuit and ``xdata`` is a list of
            gate repetitions (number of u2 gates)
    """
    #change this as required to *2
    xdata = np.arange(max_reps)
    qr = qiskit.QuantumRegister(max(qubits)+1)
    cr = qiskit.ClassicalRegister(len(qubits))
    circuits = []
    #Now using X90's RAW
    for circ_index, circ_length in enumerate(xdata):
        circ = qiskit.QuantumCircuit(qr, cr)
        circ.name = 'ampcal1Qcircuit_' + str(circ_index) + '_0'
        for qind, qubit in enumerate(qubits):
            circ.rx(np.pi/2 + dth, qr[qubit])
            for _ in range(circ_length):
                circ.barrier(qr[qubit])
                circ.rx(np.pi/2 + dth, qr[qubit])

        for qind, qubit in enumerate(qubits):
            circ.measure(qr[qubit], cr[qind])
        circuits.append(circ)

    return circuits, xdata


In [ ]:
circs, xdata = ampcal_circuits(300, [0], 0.01)
    

In [ ]:
# Set the simulator
# Add a rotation error
qubits=[0]
max_reps_sim = 75
angle_err = 0.1

err_unitary = np.zeros([2,2],dtype=complex)

for i in range(2):
    err_unitary[i,i] = np.cos(angle_err)
    err_unitary[i,(i+1) % 2] = np.sin(angle_err)
err_unitary[0,1] *= -1.0

error = coherent_unitary_error(err_unitary)
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, 'u2')
circs, xdata = ampcal_1Q_circuits(max_reps_sim, qubits)


In [ ]:
shots = 8192
circ_batch = [circs[75*i:75*(i+1)] for i in range(int(max_reps_sim/75))]
backend_result=[]
for i in circ_batch:
    backend_result.append(qiskit.execute(i, backend=QasmSimulator(), shots=shots).result())

In [ ]:
total_results = [item for sublist in map(lambda x: x.get_counts(), backend_result) for item in sublist]
prob1 = []
for i in total_results:
    try:
        prob1.append(i['1']/8192)
    except:
        prob1.append(0)
fft = np.fft.fft(prob1)
#plt.scatter(np.fft.fftfreq(len(fft)), list(map(np.abs, fft)))
plt.scatter(xdata, prob1)

In [ ]:
dict(zip(np.fft.fftfreq(len(fft)), list(map(np.abs, fft))))